# Closures
## Capturing Variables
In most languages with closures, garbage collection plays an important role. For example, consider this JavaScript code:
```Rust
// Start an animation that rearranges the rows in a table of cities.
function startSortingAnimation(cities, stat) {
    // Helper function that we'll use to sort the table. 
    // Note that this function refers to stat.
    function keyfn(city) {
        return city.get_statistic(stat); 
    }
    if (pendingSort) pendingSort.cancel();
    // Now kick off an animation, passing keyfn to it. 
    // The sorting algorithm will call keyfn later. 
    pendingSort = new SortingAnimation(cities, keyfn);
}
```
The closure `keyfn` is stored in the new `SortingAnimation` object. It’s meant to be called after `startSortingAnimation` returns. Now, normally when a function returns, all its variables and arguments go out of scope and are discarded. But here, the JavaScript engine must keep `stat` around somehow, since the closure uses it. Most JavaScript engines do this by allocating stat in the heap and letting the garbage collector reclaim it later.

Rust doesn’t have garbage collection. How will this work? To answer this question, we’ll look at two examples.

### Closures That Borrow
```Rust
/// Sort by any of several different statistics.
fn sort_by_statistic(cities: &mut Vec<City>, stat: Statistic) { 
    cities.sort_by_key(|city| -city.get_statistic(stat));
}
```

实际上，这个例子有问题，sort_by_key接受一个FnMut的参数，能执行例子中的代码的前提是，Statistic实现了Copy trait，书本中并未说明。

In this case, when Rust creates the closure, it automatically borrows a reference to stat. It stands to reason: the closure refers to stat, so it must have a reference to it.
The rest is simple. The closure is subject to the rules about borrowing and lifetimes that we described in **Chapter 5**. In particular, since the closure contains a reference to stat, Rust won’t let it outlive stat. Since the closure is only used during sorting, this example is fine.

In short, Rust ensures safety by using lifetimes instead of garbage collection. Rust’s way is faster: even a fast GC allocation will be slower than storing stat on the stack, as Rust does in this case.

来看看别的语言是如何处理闭包的：
```javascript
// Start an animation that rearranges the rows in a table of cities.
function startSortingAnimation(cities, stat) {
    // Helper function that we'll use to sort the table. 
    // Note that this function refers to stat.
    function keyfn(city) {
        return city.get_statistic(stat); 
    }
    if (pendingSort) 
        pendingSort.cancel();
    // Now kick off an animation, passing keyfn to it. 
    // The sorting algorithm will call keyfn later. 
    pendingSort = new SortingAnimation(cities, keyfn);
}
```

在这里，闭包存储在了`new SortingAnimation`这个对象中。一般来说，离开函数作用域后，变量就会被GC回收。但是此处，为了保全这个闭包，js会在堆上开辟空间让这个变量苟活，使得这个变量稍后再被GC回收。


## Function and Closure Types
```Rust
/// Sort by any of several different statistics.
fn sort_by_statistic(cities: &mut Vec<City>, stat: Statistic) { 
    cities.sort_by_key(|city| -city.get_statistic(stat));
}
```
紧接着上面这个例子，其实 任何type `fn(&City) -> i64` 都可以作为 `sort_by_key` 的参数。这是因为："In addition, all safe function pointers implement Fn, FnMut, and FnOnce, because these traits are specially known to the compiler." —— https://doc.rust-lang.org/std/primitive.fn.html

看到了吗，这里其实可以变相地说明组合相对于继承的优越性。如果是继承，你就得继承如此之多的东西，代码文本也会紧紧耦合。

所以，以下实现也是没有问题的：

```Rust
fn city_population_descending(city: &City) -> i64 { 
    -city.population
}

cities.sort_by_key(|city| -city.get_statistic(stat));
```

In [4]:
struct City {
    name: String,
    population: i64,
    country: String,
}

fn city_population_descending(city: &City) -> i64 { 
    -city.population
}

impl City {
    fn new(name: &str, population: i64, country: &str) -> Self {
        Self {
            name: name.to_string(),
            population,
            country: country.to_string(),
        }
    }
}

let mut cities = vec![
    City::new("Dubai", 3_137_000, "United Arab Emirates"),
    City::new("Hangzhou", 6_242_000, "China"),
    City::new("Lima", 10_069_000, "Peru"),
    City::new("Sofia", 1_204_000, "Bulgaria"),
    City::new("Calgary", 1_285_000, "Canada"),
    City::new("Rome", 2_627_000, "Italy"),
    City::new("Cape Town", 3_740_000, "South Africa"),
    City::new("Moscow", 12_537_000, "Russia"),
    City::new("Shanghai", 34_000_000, "China"),
    City::new("Tokyo", 38_050_000, "Japan"),
];

cities.sort_by_key(city_population_descending);

for city in cities {
    println!("{:20} {:>12}k", city.name, city.population / 1000);
};

Tokyo                       38050k
Shanghai                    34000k
Moscow                      12537k
Lima                        10069k
Hangzhou                     6242k
Cape Town                    3740k
Dubai                        3137k
Rome                         2627k
Calgary                      1285k
Sofia                        1204k


**Closure is callable, but closure is not fn**.

In fact, every closure you write has its own type, because a closure may contain data: values either borrowed or stolen from enclosing scopes. This could be any number of variables, in any combination of types. So every closure has an ad hoc type created by the compiler, large enough to hold that data. No two closures have exactly the same type. But every closure implements an Fn trait.

Since every closure has its own type, code that works with closures usually needs to be generic.

## Closure Performance
Rust’s closures are designed to be fast: faster than function pointers, fast enough that you can use them even in red-hot, performance-sensitive code. If you’re familiar with C++ lambdas, you’ll find that Rust closures are just as fast and compact, but safer.

Rust closures have none of these performance drawbacks. They’re not garbage collected. Like everything else in Rust, they aren’t allocated on the heap unless you put them in a Box, Vec, or other container. And since each closure has a distinct type, whenever the Rust compiler knows the type of the closure you’re calling, it can inline the code for that particular closure. This makes it OK to use closures in tight loops.

![image.png](asset/ch14/1.png)

* Closure (a) uses both variables. Apparently we’re looking for cities that have both tacos and tornadoes. In memory, this closure looks like a small struct containing references to the variables it uses.

    **Note that it doesn’t contain a pointer to its code!** That’s not necessary: as long as Rust knows the closure’s type, it knows which code to run when you call it.

* Closure (b) is exactly the same, except it’s a move closure, so it contains values instead of references.

* Closure (c) doesn’t use any variables from its environment. The struct is empty, so this closure does not take up any memory at all.

As the figure shows, these closures don’t take up much space. But even those few bytes are not always needed in practice. Often, the compiler can inline all calls to a closure, and then even the small structs shown in this figure are optimized away.

## Closures and Safety
### Closures That Kill


In [9]:
let my_str = "hello".to_string(); 
{
    let f = || drop(my_str);
    f();
    // f();
};

### FnOnce
```Rust
// Pseudocode for `Fn` and `FnOnce` traits with no arguments. 
trait Fn() -> R {
    fn call(&self) -> R; 
}

trait FnOnce() -> R {
    fn call_once(self) -> R;
}
```

For an `Fn` closure, `closure()` expands to `closure.call()`. This method takes self by reference, so the closure is not moved. But if the closure is only safe to call once, then `closure()` expands to `closure.call_once()`. That method takes self by value, so the closure is used up.

其实rust在这里是比较讨厌的，rust在闭包设计上，类型传递不够透明，比如实现了`Copy`的类型，不用`move`或`&`就会自动copy入场，而没有实现`Copy`的对象，则会直接所有权转移。（好吧，后面有一个小节讲了，亏我研究了好一会）

### FnMut
Rust considers non-mut values safe to share across threads. But it wouldn’t be safe to share non-mut closures that contain mut data: calling such a closure from multiple threads could lead to all sorts of race conditions as multiple threads try to read and write the same data at the same time.

Therefore, Rust has one more category of closure, FnMut, the category of closures that write. FnMut closures are called by mut reference, as if they were defined like this:
```Rust
trait FnMut() -> R {
    fn call_mut(&mut self) -> R;
}
```
Any closure that requires mut access to a value, **but doesn’t drop any values**, is an
FnMut closure.

Every `Fn` meets the requirements for `FnMut`, and every `FnMut` meets the requirements for `FnOnce`.

Instead, `Fn()` is a subtrait of `FnMut()`, which is a subtrait of `FnOnce()`.
![image.png](asset/ch14/2.png)

### Copy and Clone for Closures

Just as Rust automatically figures out which closures can be called only once, it can figure out which closures can implement `Copy` and `Clone`, and which cannot.

As we explained earlier, closures are represented as structs that contain either the values (for move closures) or references to the values (for non-move closures) of the variables they capture. The rules for `Copy` and `Clone` on closures are just like the `Copy` and `Clone` rules for regular structs. A non-move closure that doesn’t mutate variables holds only shared references, which are both `Clone` and `Copy`, so that closure is both `Clone` and `Copy` as well:

In [12]:
let y = 10;
{
    let add_y = |x| x+y;
    let copy_of_add_y = add_y; // This closure is `Copy`, so... 
    assert_eq!(add_y(copy_of_add_y(22)), 42); // ... we can call both.
};

On the other hand, a non-move closure that does mutate values has mutable references within its internal representation. Mutable references are neither `Clone` nor `Copy`, so neither is a closure that uses them:

下面这个例子很重要，细品，我加了几个例子，使得这个机制更直白

In [27]:
let mut x = 0;
{
    let mut add_to_x = |n| { x += n; x };
    let mut copy_of_add_to_x = add_to_x; // this moves, rather than copies 
    // assert_eq!(add_to_x(copy_of_add_to_x(1)), 2); // error: use of moved value
    // assert_eq!(add_to_x(1), 1); // error: use of moved value
    // assert_eq!(copy_of_add_to_x(copy_of_add_to_x(1)), 2); // error: borrow `copy_of_add_to_x` as mutable more than once at a time
    assert_eq!(copy_of_add_to_x(1), 1);
    assert_eq!(copy_of_add_to_x(2), 3);
};

For a move closure, the rules are even simpler. If everything a `move` closure captures is `Copy`, it’s `Copy`. If everything it captures is `Clone`, it’s `Clone`. For instance:

In [31]:
{
    let mut greeting = String::from("Hello, "); 
    let greet = move |name| {
        greeting.push_str(name);
        println!("{}", greeting);
    };
    greet.clone()("Alfred");
    greet.clone()("Bruce");
};

Hello, Alfred
Hello, Bruce


This `.clone()(...)` syntax is a little weird, but it just means that we clone the closure and then call the clone.

## Callbackks 
本小节建议全文通读